In [1]:
%cd ..

/Users/ishamrashik/Desktop/machine-learning-and-ai-projects/community-mcp-session/aws_community_day/youtube-sentiment-insights


In [2]:
import mlflow
import os 

# Set AWS credentials (replace with your actual credentials)
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION')

# Then set MLflow tracking URI
mlflow.set_tracking_uri("http://3.29.137.20:5000")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

2025/10/12 11:19:57 INFO mlflow.tracking.fluent: Experiment with name 'Exp 4 - Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-5fb77f1c/985419319765511025', creation_time=1760253597974, experiment_id='985419319765511025', last_update_time=1760253597974, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [6]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [7]:
from utilities import load_data, INTERIM_DATA_PATH

In [8]:
train_df_path = os.path.join(INTERIM_DATA_PATH, "train_processed.csv")
val_df_path = os.path.join(INTERIM_DATA_PATH, "val_processed.csv")
test_df_path = os.path.join(INTERIM_DATA_PATH, "test_processed.csv")

train_df = load_data(train_df_path)
val_df = load_data(val_df_path)
test_df = load_data(test_df_path)

2025-10-12 11:23:46,012 - model_building - DEBUG - Data loaded from data/interim/train_processed.csv
2025-10-12 11:23:46,018 - model_building - DEBUG - Data loaded from data/interim/val_processed.csv
2025-10-12 11:23:46,025 - model_building - DEBUG - Data loaded from data/interim/test_processed.csv


In [9]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 10000  # Set max_features to 1000 for TF-IDF

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    # Step 1: Vectorize the text comments (Bag of Words)
    X_train_text = vectorizer.fit_transform(train_df['clean_comment']).toarray()
    X_val_text = vectorizer.transform(val_df['clean_comment']).toarray()

    # Step 2: Extract numerical features
    # Define the numerical feature columns to include
    numerical_features = ['word_count', 'num_stop_words', 'num_chars', 'num_chars_cleaned']

    X_train_numerical = train_df[numerical_features].values
    X_val_numerical = val_df[numerical_features].values

    # Step 3: Combine text features with numerical features
    import numpy as np

    X_train = np.hstack([X_train_text, X_train_numerical])
    X_val = np.hstack([X_val_text, X_val_numerical])

    # Target variable
    y_train = train_df['category']
    y_val = val_df['category']

    # Step 3: Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (only apply to the training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train, y_train = smote.fit_resample(X_train, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train, y_train = adasyn.fit_resample(X_train, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train, y_train = rus.fit_resample(X_train, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train, y_train = smote_enn.fit_resample(X_train, y_train)

    # Step 5: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_val)

        # Log accuracy
        accuracy = accuracy_score(y_val, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_val, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_val, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)


2025/10/12 11:26:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:26:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://3.29.137.20:5000/#/experiments/985419319765511025/runs/94b1442b01b140d09e2fca4ca536df33
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/985419319765511025


2025/10/12 11:27:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:27:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://3.29.137.20:5000/#/experiments/985419319765511025/runs/16babf7f3f4846c890ecb03ea9cd54c8
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/985419319765511025


2025/10/12 11:27:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:28:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://3.29.137.20:5000/#/experiments/985419319765511025/runs/9f39614f43e34634bef139d444739107
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/985419319765511025


2025/10/12 11:28:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:28:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://3.29.137.20:5000/#/experiments/985419319765511025/runs/9a20d472a2ae4062a0e15b83b48edf79
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/985419319765511025


2025/10/12 11:28:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 11:28:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://3.29.137.20:5000/#/experiments/985419319765511025/runs/8dc1cf66f19642cd94c4abbe21080d82
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/985419319765511025
